In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',1500)

In [2]:
df = pd.read_csv("testing_reviews.csv")

In [3]:
print(df.isnull().sum())
print(len(pd.unique(df["job_location"])))
print(len(pd.unique(df["company_name"]))) #2.6 for the larger df
# print(len(pd.unique(df["Industry"])))
# print(len(pd.unique(df["Type"])))
# print(len(pd.unique(df["Size"])))
df.columns

company_cons     312
company_info     311
company_name       0
company_pros     311
description        4
job_location     311
job_title          0
outlook          311
post_date          0
rating             0
recommend        311
salary_est      4020
salary_high     4020
salary_low      4020
dtype: int64
265
2646


Index(['company_cons', 'company_info', 'company_name', 'company_pros',
       'description', 'job_location', 'job_title', 'outlook', 'post_date',
       'rating', 'recommend', 'salary_est', 'salary_high', 'salary_low'],
      dtype='object')

In [4]:
df = pd.read_csv("reviews.csv")
# df

In [5]:
from ast import literal_eval
# print(100*df["salary_est"].isnull().sum()/len(df["salary_est"]),"%")

df = df[(df["company_name"]!="*** ERROR - NO COMPANY LISTED. SEE DESCRIPTION FOR POSSIBLE HINTS *** [XX]") & (df["company_info"].isnull()!=True)]
print((df["company_name"]=="*** ERROR - NO COMPANY LISTED. SEE DESCRIPTION FOR POSSIBLE HINTS *** [XX]").sum())
print(df["company_info"].isnull().sum())
# print(df["company)info"].contains(">").sum())



0
0


In [6]:
df["company_info"] = df["company_info"].apply(literal_eval)
# df["company_info"]

df["company_info"] = pd.Series([dict(i) for i in df["company_info"]])
# df["company_info"]
# df["company_info"].items

In [7]:
df = pd.concat([df.drop('company_info', axis = 1), df["company_info"].apply(pd.Series)], axis = 1)
#C:\Users\mjgol\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:77: RuntimeWarning: '<' 
# not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
#   result = result.union(other)

In [8]:
# df

In [9]:
df.isnull().sum()
print(len(pd.unique(df["job_location"])))
print(len(pd.unique(df["company_name"]))) #well that isn't a lot 
print(len(pd.unique(df["Industry"])))
print(len(pd.unique(df["Type"])))
print(len(pd.unique(df["Size"])))

df.columns

216
1709
38
463
8


Index(['company_cons', 'company_name', 'company_pros', 'description',
       'job_location', 'job_title', 'outlook', 'post_date', 'rating',
       'recommend', 'salary_est', 'salary_high', 'salary_low', 'Competitors',
       'Founded', 'Headquarters', 'Industry', 'Now known as ', 'Part of ',
       'Revenue', 'Size', 'Type'],
      dtype='object')

In [10]:
for i in np.where(df["Part of "]==""):
    df.loc[i, "Founded"] = df.loc[i,"Type"]
    df.loc[i,"Type"] = df.loc[i,"Industry"]
    df.loc[i,"Industry"] = df.loc[i,"Revenue"]
    df.loc[i,"Revenue"] = df.loc[i,"Competitors"]
    df.loc[i, "Part of "] = np.nan
    df.loc[i,"Competitors"] = np.nan
#     type to founded
#     revenue to industry
#     industry to type
#     competitors to revenue
    

In [11]:
# df

In [12]:
df.isnull().sum()

company_cons        1
company_name        0
company_pros        0
description         0
job_location        0
job_title           0
outlook             0
post_date           0
rating              0
recommend           0
salary_est          0
salary_high         0
salary_low          0
Competitors      2127
Founded           100
Headquarters       10
Industry           60
Now known as     8473
Part of          8503
Revenue           125
Size                0
Type                0
dtype: int64

In [13]:
# df.drop(["Now known as ", "Part of ", "post_date"], axis = 1)

In [14]:
# distinct_df = distinct
#Greentech media data analyst - 	Brooklyn, 061 for HQ

In [15]:
len(df["Type"])

8503

In [17]:
distinct_df = df.drop_duplicates(["company_name", "description", "job_location", "job_title"])
distinct_df = distinct_df.drop(["company_cons", "company_pros", "outlook", "recommend", "Headquarters"], axis = 1)

In [18]:
distinct_df["description"][5]
print(len(distinct_df["job_title"]))
# df[df.C.str.contains("XYZ") == False]
distinct_df = distinct_df[distinct_df["job_title"].str.contains("[Ii]ntern(?!a)")==False] 
len(distinct_df)
#to prevent internal, or international, etc.
#we also don't want internships

1712


1664

In [184]:
test = distinct_df["description"]
test.reset_index(drop=True)
test.values
print(type(test.values))
# [re.search("(.{9})years?",i) for i in test.values]
# [re.findall("(.{20}) (YEARS?) (.{20})",i,re.IGNORECASE) for i in test.values]
# [re.findall("(.{20})(B\.?A\.?(?!\w)|B\.?S\.?+)(.{20})",i, re.IGNORECASE) for i in test.values]

##WILL NEED SANITY CHECK ON THIS
##EDUCATION LISTINGS
bachelors = [re.findall("(?<![A-Z])B\.?S\.?c?(?![A-Z])|(?<![A-Z])B\.?A\.?(?![A-Z])|BACHELOR|UNDERGRAD.{0,40} DEGREE|ASSOCIATE'?S?.{20}DEGREE",i, re.IGNORECASE) for i in test.values]
bachelors
print(len([x for x in bachelors if x]))
masters = [re.findall("(MASTER'?S?.{0,40}DEGREE|GRADUATE.{0,40}DEGREE|(?<![A-Z])M\.?S\.?(?![A-Z]|\sDYNAMICS|,\sDSC)(?!-?~?\s?OFFICE|\sEXCEL|\sWORD|\sACCESS|-?\s?SQL)|ADVANCED?.{0,40}DEGREE)",i,re.IGNORECASE) for i in test.values]
masters
mba = [re.findall("([\s|-|/]MBA[\s|-|/]|[\s|-|/]MBUS[\s|-|/]|[\s|-|/]MBS[\s|-|/]|MASTERS? OF BUSINESS)",i,re.IGNORECASE) for i in test.values]
mba
print(len([x for x in mba if x]))
print(len([x for x in masters if x]))
phd = [re.findall("(PH\.?D|ADVANCED?.{0,40}DEGREE|DOCTORA[TE|L]|POST-?\s?GRADUATE)",i,re.IGNORECASE) for i in test.values]
phd
print(len([x for x in phd if x]))

#WHAT THE OTHER PERSON DID
#  edu_dict = Counter({'Bachelor':doc_frequency['bachelor'],'Master':doc_frequency['master'],
#                           'PhD': doc_frequency['phd'],'MBA':doc_frequency['mba']})


<class 'numpy.ndarray'>
837
35
693
565


In [217]:
python = [re.findall("[\s,-.]PYTHON[\s,-.]",i,re.IGNORECASE) for i in test.values]
python
print(len([x for x in python if x]))
R = [re.findall("[\s,-]R[\s,-\.]",i,re.IGNORECASE) for i in test.values]
print(len([x for x in R if x]))
SQL =[re.findall("[\s,-]SQL[\s,-.]",i,re.IGNORECASE) for i in test.values]
print(len([x for x in SQL if x]))
java = [re.findall("JAVA",i,re.IGNORECASE) for i in test.values]
print(len([x for x in java if x]))
C = [re.findall("[\s,-.]C[\s,-\.|++]",i,re.IGNORECASE) for i in test.values]
print(len([x for x in C if x]))

714
524
719
402
177


In [ ]:
print("HOW TO GET CITY INFORMATION")

In [ ]:
cities = ["boston","new-york","washington","san-jose","los-angeles","chicago","austin","huntsville","seattle","denver","san-francisco","miami","tampa","palo-alto","houston","atlanta"]
city_ids = [1154532,1132348,1138213,1147436,1146821,1128808,1139761,1127653,1150505,1148170,1147401,1154170,1154429,1147434,1140171,1155583 ]


In [ ]:
    for city_id in city_ids:
        for i in range(1,31):
            pass
#             print("https://www.glassdoor.com/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC"+
#                   str(city_id)+
#                   "_KO9,23_IP"+
#                   str(i)+
#                   ".htm")

In [ ]:
x =[("https://www.glassdoor.com/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC" + 
    str(city_id) +
    "_KO9,23_IP" +
    str(num) + 
    ".htm") for city_id in city_ids for num in range(1,31)]
# x
#[x for y in collection for x in y] # [A for B in C for D in E]
#for y in collection:     #      for B in C:
#    for x in y:          #          for D in E: (in this case: for A in B)
        # receive x      #              # receive A


In [ ]:
# [str(len(city)) for city in cities]
# [str(len(city)+14) for city in cities]

In [ ]:
cities = list(zip(cities,city_ids))
cities #this has the right line up
start_url = [("https://www.glassdoor.com/Job/"+
  city[0]+
  "-data-scientist-jobs-SRCH_IL.0,"+
  str(len(city[0]))+
  "_IC"+
  str(city[1])+
  "_KO"+
  str(len(city[0])+1)+
  ","+
  str(len(city[0])+15)+
  "_IP"+
  str(i) +
  ".htm") for city in cities for i in range(1,31)]
start_url

cities[1][0]

#PROPER SAN JOSE: https://www.glassdoor.com/Job/san-jose-data-scientist-jobs-SRCH_IL.0,8_IC1147436_KO9,23_IP2.htm

In [ ]:
x = [url + "_KO9,23_IP" for url in x]
# x

In [ ]:
i = (range(1,31))
# [(url + str(num) + ".htm") for url in x for num in i]

In [ ]:
 seq_x = [1, 2, 3, 4]
seq_y = 'abc'
[(x,y) for x in seq_x for y in seq_y]
[(1, 'a'), 
 (1, 'b'), 
 (1, 'c'), 
 (2, 'a'), 
 (2, 'b'), 
 (2, 'c'), 
 (3, 'a'), 
 (3, 'b'), 
 (3, 'c'), 
 (4, 'a'), 
 (4, 'b'), 
 (4, 'c')]

In [ ]:
["https://www.glassdoor.com/Job/new-york-data-scientist-jobs-SRCH_IL.0,8_IC"+str(city_id)+"_KO9,23_IP"+str(i)+".htm" for city_id for i in range(1,31) for city_id in city_ids]


In [ ]:
if not all(item.values()):
	raise DropItem("Missing values!")